# Text Preprocessing for EBC Feedback Sentiment Analysis

# Imports and Constants

In [1]:
import pandas as pd
import re
import nltk
import os
import csv
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import string
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer # this is for the built-in polarity extraction
import nltk
nltk.download('vader_lexicon')

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
DATA_FILE_PATH = 'C:\\Users\\Yostina\\SentimentNLP\\'
CLEAN_DATA_FILE_NAME = 'clean_feedback_data.csv'
SAVE_FILE = True
TOKENIZED_DATA_FILE_NAME = 'tokenized_feedback_data.csv'

# Load Data

In [3]:
df = pd.read_csv(DATA_FILE_PATH + CLEAN_DATA_FILE_NAME)

In [4]:
pd.set_option('display.max_colwidth', None)
df.head()

,Region,Sex,Age,Edu_Level,Feedback,Fb_Day,Fb_Month,Fb_Year
0,Debub,M,50,Cert,ኢቲቪ ሰራተኞች በሙሉ እንኳን ለገና በአል በሰላም አደረሳችሁ፡፡,1,1,2008.0
1,Debub,M,50,Cert,መንግስት በአገራችን በደረሰው የተለያዩ ችግሮች ላይ እየወሰደ ያለው እርምጃ በጣም ጥሩ ነው በዚሁ ይቀጥል፡፡,1,1,2008.0
2,Debub,M,50,Cert,ኢቲቪ ደግሞ በዜና በወቅታዊ ጉዳዮች ላይ እውነት ለመናገር በጣም ጥሩ እየሰራችሁ ነው በርቱ፡፡,1,1,2008.0
3,Debub,M,50,Cert,ኢቲቪ ቢሰራ የምንለው ህዝቡ ስሩን የሰደደው የህዋት አስተዳደር አሁንም ባሉት አመራሮች እየተተገበረ ስለሆነ ወደታች ወርዳችሁ ብትሰሩ የበለጠ ተሰሚ ትሆናለችሁ አገርንም በሰላም ታስቀጥላላችሁ ሀላፊነታችሁን ተወጡ፡፡,1,1,2008.0
4,Debub,M,30,Sec. Sc,ኢቲቪ ዜና 57 ላይ የሚቀርቡት ወቅታዊ ዜናዎች በሀገርና በውጭ በጣም ጥሩ መረጃ እያገኝን ነው፡፡,1,1,2008.0


# Tokenize feedback

Use the specialized NLTK TweetTokenizer to keep hashtags and emojis 

In [5]:
tknzr = TweetTokenizer()

In [6]:
df['tokens'] = df['Feedback'].apply(tknzr.tokenize)

In [7]:
df.iloc[550:600][['Feedback', 'tokens']]

,Feedback,tokens
550,በኢቲቪ እየቀረበ ያለው ጤናዎ በቤትዎ ላይ በትምህርትና በኮረና የቀረበው ወይት ጥሩ ነው ፡፡,"[በኢቲቪ, እየቀረበ, ያለው, ጤናዎ, በቤትዎ, ላይ, በትምህርትና, በኮረና, የቀረበው, ወይት, ጥሩ, ነው, ፡, ፡]"
551,በኢቲቪ እንደዚህ አይነት ፕሮግራሞች ስታቀርቡ ለምን ህብረተሰቡን የሚያሳትፍ ስልክ አስተያየት መስጫ ብኖር ጥሩ ነው፡፡,"[በኢቲቪ, እንደዚህ, አይነት, ፕሮግራሞች, ስታቀርቡ, ለምን, ህብረተሰቡን, የሚያሳትፍ, ስልክ, አስተያየት, መስጫ, ብኖር, ጥሩ, ነው, ፡, ፡]"
552,ኢቲቪ ፈጣን መረጃ በማድረስ እና እድሜ ጠገብ አገራዊ የህዝብ ሚዲያ ስለሆነ በጣም እንወደዋለን፡፡,"[ኢቲቪ, ፈጣን, መረጃ, በማድረስ, እና, እድሜ, ጠገብ, አገራዊ, የህዝብ, ሚዲያ, ስለሆነ, በጣም, እንወደዋለን, ፡, ፡]"
553,ኢቲቪ በኮረና ላይ ለህብረተሰቡ ጥሩ ግንዛቤ ሰታችኋል ነግር ግን ቢሰራ የምንለ አሁንም ህዝቡ ተዘናግቷል፡፡,"[ኢቲቪ, በኮረና, ላይ, ለህብረተሰቡ, ጥሩ, ግንዛቤ, ሰታችኋል, ነግር, ግን, ቢሰራ, የምንለ, አሁንም, ህዝቡ, ተዘናግቷል, ፡, ፡]"
554,ሚዲያው ተቋማት ጥራት የመሳሪያ አቅርቦት በአግባቡ ተደራሽነት ችግር አለ ማቀባበያ መሳሪያዎችን ችግር ከመፍታት በተለይ በገጠሪቱ አካባቢዎች ቢታሰብበት ጥሩ ነው፡፡,"[ሚዲያው, ተቋማት, ጥራት, የመሳሪያ, አቅርቦት, በአግባቡ, ተደራሽነት, ችግር, አለ, ማቀባበያ, መሳሪያዎችን, ችግር, ከመፍታት, በተለይ, በገጠሪቱ, አካባቢዎች, ቢታሰብበት, ጥሩ, ነው, ፡, ፡]"
555,ኢቲቪ የሚቀርቡ ዜናዎች ጥራትና ለውጥ አለው ጥሩ መረጃ በፍጥነትና እወነትን የተመረኮዘ ዜና እያቀረባችሁ ነው በዚሁ ይቀጥል፡፡,"[ኢቲቪ, የሚቀርቡ, ዜናዎች, ጥራትና, ለውጥ, አለው, ጥሩ, መረጃ, በፍጥነትና, እወነትን, የተመረኮዘ, ዜና, እያቀረባችሁ, ነው, በዚሁ, ይቀጥል, ፡, ፡]"
556,ኢቲቪ የሚቀርወ ወይም ድክምቱ የህዝብ ሚዲያ እንደመሆኑ ወርዶ ከመስራት በጣም ይቀረዋል በዚህ ላይ ቢሰራ፡፡,"[ኢቲቪ, የሚቀርወ, ወይም, ድክምቱ, የህዝብ, ሚዲያ, እንደመሆኑ, ወርዶ, ከመስራት, በጣም, ይቀረዋል, በዚህ, ላይ, ቢሰራ, ፡, ፡]"
557,ኢቲቪ ነፃ መስመር ሰቶ በነፃ ሀሳብ እንድንሰጥ መደረጉ በጣም ጠሩ ነው እናመሰግናልን፡፡,"[ኢቲቪ, ነፃ, መስመር, ሰቶ, በነፃ, ሀሳብ, እንድንሰጥ, መደረጉ, በጣም, ጠሩ, ነው, እናመሰግናልን, ፡, ፡]"
558,መንግስት የወሰደው የብር ለውጥ ውሳኔ ወቅታዊና አስፈላጊ ነው ምክንያቱም የአገርን ሰላም ችግር ይፈታል ብለን እናምናለን፡፡,"[መንግስት, የወሰደው, የብር, ለውጥ, ውሳኔ, ወቅታዊና, አስፈላጊ, ነው, ምክንያቱም, የአገርን, ሰላም, ችግር, ይፈታል, ብለን, እናምናለን, ፡, ፡]"
559,ኢቲቪ የሚያቀርበው ወቅታዊ ትኩስ ዜናዎች በጣም ጠሩ መረጃ እያገኘን ነው በዚሁ ይቀጥል፡፡,"[ኢቲቪ, የሚያቀርበው, ወቅታዊ, ትኩስ, ዜናዎች, በጣም, ጠሩ, መረጃ, እያገኘን, ነው, በዚሁ, ይቀጥል, ፡, ፡]"


# Save Cleaned and Tokenized Data

In [8]:
if SAVE_FILE:
    df.to_csv(DATA_FILE_PATH + TOKENIZED_DATA_FILE_NAME, index=False,encoding="utf-8")

In [9]:
#function that removes all punctuations. helps in creation of set of words
def remov_punct(withpunct):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    without_punct = ""
    char = 'nan'
    for char in withpunct:
        if char not in punctuations:
            without_punct = without_punct + char
    return(without_punct)

In [10]:
#function to identify Sentiment classification 
def findpolar(test_data):
    foundpolar=''
    sia = SentimentIntensityAnalyzer()
    sia.lexicon.update({u'97.1': -3.10,u'ኤፍ ኤም': -2.00,u'FM': -7.20,u'ኢ.ሬ':5.2,u'ማለዳ':5.2,u'ዘፈን':4.2,u'ይጋብዙ':4.2,u'ማዐድ':4.2}) 
    polarity = sia.polarity_scores(test_data)["compound"];
    
    if(polarity >= 0.1):
     foundpolar = 'NR'
    if(polarity <= -0.1):
       foundpolar = 'FM'  
    if(polarity>= -0.1 and polarity<= 0.1):
     foundpolar = 'ETV'
     
    return(foundpolar)

In [11]:
catdata = pd.read_csv("C://Users//Yostina//SentimentNLP//tokenized_feedback_data.csv",encoding="utf-8")       
catdata.head()          
#create a new variable called sentiment
catdata['category'] = '';
for s in range(len(catdata)):
    comment = remov_punct(catdata['Feedback'][s])
    categorys = findpolar(comment)
    catdata['category'][s] = categorys  
#output the sentiment mapped data            
da=catdata.to_csv("C://Users//Yostina//SentimentNLP//sentiment_mapped_datas1.csv",index = False)

<ipython-input-11-65408ca0fea5>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  catdata['category'][s] = categorys


In [12]:
#function to identify Sentiment classification 
def findpolar(test_data):
    foundpolar=0
    sia = SentimentIntensityAnalyzer()
    sia.lexicon.update({u'ጥሩ': 3.1,u'ዘረኛ': 5.0,u'አሪፍ': 2.0,u'መደምሰስ': -2.0,u'ይጨፈጨፋል': -2.0,u'ችግር': -2.0,u'አሳዝኖናል': -2.0,u' አይደለም': -2.1,u'ተቋርጧል':-1.0,u'ሰላም':2.2,u'ምላሽ':-2.0,u'ተአምረኛ':1.5,u'ጉዳት':-2.1,u'የሚያወላውል':-2.5,u'ዱብዳ':-2.4,u'ባለፀጋ':3.2,u'ትምክህተኛ':-1.2,u'ቁጥብ':2.3,u'ጎሽ':3.2,u'እድል':2.5,u'ተመፃዳቂ':-2.3,u'አርበኛ':3.1,u'ልክ':4.1,u'ፍጭት':-2.5,u'ጠቢብ':2.1,u'ያልሰመረ':-2.5,u'አቤቱታ':-1.2,u'ፍቃድ':2.5,u'ውሸት':-1.1,u'ግርማ':2.1,u'ዘለቄታ':2,u'አውደልዳይ':-1.2,u'ርህራሄ':2.0,u'አሜን':1.8,u'ተብታባ':-2.4}) 
    polarity = sia.polarity_scores(test_data)["compound"];
    if(polarity >= 0.1):    
     foundpolar = 1
    if(polarity <= -0.1):
     foundpolar = -1 
    if(polarity>= -0.1 and polarity<= 0.1):
     foundpolar = 0
     
    return(foundpolar)

In [13]:
catdata = pd.read_csv("C://Users//Yostina//SentimentNLP//sentiment_mapped_datas1.csv",encoding="utf-8")       
#catdata.head()          
#create a new variable called sentiment
catdata['sentiment'] = '';
for s in range(len(catdata)):
    comment = remov_punct(catdata['Feedback'][s])
    sentiments = findpolar(comment)
    catdata['sentiment'][s] = sentiments  
#output the sentiment mapped data            
da=catdata.to_csv("C://Users//Yostina//SentimentNLP//sentiment_mapped_datas4.csv",index = False)               


<ipython-input-13-fd9e74f91dcc>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  catdata['sentiment'][s] = sentiments


In [14]:
catdata.head()

,Region,Sex,Age,Edu_Level,Feedback,Fb_Day,Fb_Month,Fb_Year,tokens,category,sentiment
0,Debub,M,50,Cert,ኢቲቪ ሰራተኞች በሙሉ እንኳን ለገና በአል በሰላም አደረሳችሁ፡፡,1,1,2008.0,"['ኢቲቪ', 'ሰራተኞች', 'በሙሉ', 'እንኳን', 'ለገና', 'በአል', 'በሰላም', 'አደረሳችሁ', '፡', '፡']",ETV,0
1,Debub,M,50,Cert,መንግስት በአገራችን በደረሰው የተለያዩ ችግሮች ላይ እየወሰደ ያለው እርምጃ በጣም ጥሩ ነው በዚሁ ይቀጥል፡፡,1,1,2008.0,"['መንግስት', 'በአገራችን', 'በደረሰው', 'የተለያዩ', 'ችግሮች', 'ላይ', 'እየወሰደ', 'ያለው', 'እርምጃ', 'በጣም', 'ጥሩ', 'ነው', 'በዚሁ', 'ይቀጥል', '፡', '፡']",ETV,1
2,Debub,M,50,Cert,ኢቲቪ ደግሞ በዜና በወቅታዊ ጉዳዮች ላይ እውነት ለመናገር በጣም ጥሩ እየሰራችሁ ነው በርቱ፡፡,1,1,2008.0,"['ኢቲቪ', 'ደግሞ', 'በዜና', 'በወቅታዊ', 'ጉዳዮች', 'ላይ', 'እውነት', 'ለመናገር', 'በጣም', 'ጥሩ', 'እየሰራችሁ', 'ነው', 'በርቱ', '፡', '፡']",ETV,1
3,Debub,M,50,Cert,ኢቲቪ ቢሰራ የምንለው ህዝቡ ስሩን የሰደደው የህዋት አስተዳደር አሁንም ባሉት አመራሮች እየተተገበረ ስለሆነ ወደታች ወርዳችሁ ብትሰሩ የበለጠ ተሰሚ ትሆናለችሁ አገርንም በሰላም ታስቀጥላላችሁ ሀላፊነታችሁን ተወጡ፡፡,1,1,2008.0,"['ኢቲቪ', 'ቢሰራ', 'የምንለው', 'ህዝቡ', 'ስሩን', 'የሰደደው', 'የህዋት', 'አስተዳደር', 'አሁንም', 'ባሉት', 'አመራሮች', 'እየተተገበረ', 'ስለሆነ', 'ወደታች', 'ወርዳችሁ', 'ብትሰሩ', 'የበለጠ', 'ተሰሚ', 'ትሆናለችሁ', 'አገርንም', 'በሰላም', 'ታስቀጥላላችሁ', 'ሀላፊነታችሁን', 'ተወጡ', '፡', '፡']",ETV,0
4,Debub,M,30,Sec. Sc,ኢቲቪ ዜና 57 ላይ የሚቀርቡት ወቅታዊ ዜናዎች በሀገርና በውጭ በጣም ጥሩ መረጃ እያገኝን ነው፡፡,1,1,2008.0,"['ኢቲቪ', 'ዜና', '57', 'ላይ', 'የሚቀርቡት', 'ወቅታዊ', 'ዜናዎች', 'በሀገርና', 'በውጭ', 'በጣም', 'ጥሩ', 'መረጃ', 'እያገኝን', 'ነው', '፡', '፡']",ETV,1
